In [21]:
import pandas as pd
import numpy as np
# Import Regular Expression
import re
# Loading the data
df = pd.read_csv("Data\jumia_phones.csv")
# Display the first few rows of the data to get a sense of the data structure
df.head()

,Name,Price,Old Price,Discount,Rating,Number of Reviews,Search Ranking
0,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1"
1,"Tecno Spark 20, Android 13, 6.6"", 128GB + 4GB ...","KSh 12,925","KSh 15,000",14%,4.4 out of 5,4.4 out of 5(135),"Page 1, Rank 2"
2,"Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (...","KSh 10,000",NaN,NaN,4.2 out of 5,4.2 out of 5(151),"Page 1, Rank 3"
3,"Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM ...","KSh 14,000",NaN,NaN,4.5 out of 5,4.5 out of 5(29),"Page 1, Rank 4"
4,"Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (...","KSh 10,000",NaN,NaN,4.3 out of 5,4.3 out of 5(249),"Page 1, Rank 5"


In [22]:
print("df shape:",df.shape)
print ("------------------------------------------------")
print("df missing values",df.isna().sum())
print ("------------------------------------------------")
print("df missing values",df.isna().sum())

df shape: (12000, 7)
------------------------------------------------
df missing values Name                    0
Price                   0
Old Price            3899
Discount             3899
Rating                300
Number of Reviews     300
Search Ranking          0
dtype: int64
------------------------------------------------
df missing values Name                    0
Price                   0
Old Price            3899
Discount             3899
Rating                300
Number of Reviews     300
Search Ranking          0
dtype: int64


In [174]:
#check for duplicates in df
print(df.duplicated().sum())

0


## Feature Extraction and Preprocessing

In [23]:
#duplicate the name column

df['Name_duplicate'] = df['Name']
df.head()

,Name,Price,Old Price,Discount,Rating,Number of Reviews,Search Ranking,Name_duplicate
0,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1","XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S..."
1,"Tecno Spark 20, Android 13, 6.6"", 128GB + 4GB ...","KSh 12,925","KSh 15,000",14%,4.4 out of 5,4.4 out of 5(135),"Page 1, Rank 2","Tecno Spark 20, Android 13, 6.6"", 128GB + 4GB ..."
2,"Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (...","KSh 10,000",NaN,NaN,4.2 out of 5,4.2 out of 5(151),"Page 1, Rank 3","Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (..."
3,"Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM ...","KSh 14,000",NaN,NaN,4.5 out of 5,4.5 out of 5(29),"Page 1, Rank 4","Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM ..."
4,"Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (...","KSh 10,000",NaN,NaN,4.3 out of 5,4.3 out of 5(249),"Page 1, Rank 5","Itel S23 6.6"", 128GB + 4GB RAM, 50MP Camera, (..."


In [119]:
# Get the name entries in the name column and assign them to a list called data
#select the two name columns
data = df[['Name','Name_duplicate']]

# convert the data type of data to a list

# Check the data type of data
type(data)

pandas.core.frame.DataFrame

In [120]:
#create a list of dictionaries containing name and name 2

name_list = [{'name': name,'name_duplicate':name_duplicate} for name, name_duplicate in zip(data['Name'], data['Name_duplicate'])]

name_list

[{'name': 'XIAOMI Redmi A3, 6.71", 3GB RAM + 64GB (Dual SIM), 5000mAh, Midnight Black  (2 YRs WRTY)',
  'name_duplicate': 'XIAOMI Redmi A3, 6.71", 3GB RAM + 64GB (Dual SIM), 5000mAh, Midnight Black  (2 YRs WRTY)'},
 {'name': 'Tecno Spark 20, Android 13, 6.6", 128GB + 4GB RAM(4GB Extended), 50MP, 5000mAh, Gravity Black(1YR WRTY)',
  'name_duplicate': 'Tecno Spark 20, Android 13, 6.6", 128GB + 4GB RAM(4GB Extended), 50MP, 5000mAh, Gravity Black(1YR WRTY)'},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Mystery White (1YR WRTY)',
  'name_duplicate': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Mystery White (1YR WRTY)'},
 {'name': "Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM (Dual Sim) 50MP Camera, 5000mAh, Black",
  'name_duplicate': "Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM (Dual Sim) 50MP Camera, 5000mAh, Black"},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Starry Black',
  'name

In [ ]:
#clean the first entry in the list
# Define function to clean 'name_duplicate'
def clean_name_duplicate(name_duplicate):
    # Remove unwanted characters, keeping numbers and spaces
    cleaned = re.sub(r"[,'+\-]", " ", name_duplicate)  # Replace commas, single quotes, and plus signs with space
    cleaned = re.sub(r"\s+", " ", cleaned).strip()  # Replace multiple spaces with a single space
    return cleaned
# Apply the cleaning function to 'name_duplicate' for each dictionary in the list
for entry in name_list:
    entry['name_duplicate'] = clean_name_duplicate(entry['name_duplicate'])

# Output the cleaned data

name_list[0:5]

    

[{'name': 'XIAOMI Redmi A3, 6.71", 3GB RAM + 64GB (Dual SIM), 5000mAh, Midnight Black  (2 YRs WRTY)',
  'name_duplicate': 'XIAOMI Redmi A3 6.71" 3GB RAM 64GB (Dual SIM) 5000mAh Midnight Black (2 YRs WRTY)'},
 {'name': 'Tecno Spark 20, Android 13, 6.6", 128GB + 4GB RAM(4GB Extended), 50MP, 5000mAh, Gravity Black(1YR WRTY)',
  'name_duplicate': 'Tecno Spark 20 Android 13 6.6" 128GB 4GB RAM(4GB Extended) 50MP 5000mAh Gravity Black(1YR WRTY)'},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Mystery White (1YR WRTY)',
  'name_duplicate': 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Mystery White (1YR WRTY)'},
 {'name': "Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM (Dual Sim) 50MP Camera, 5000mAh, Black",
  'name_duplicate': 'Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual Sim) 50MP Camera 5000mAh Black'},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Starry Black',
  'name_duplicate': 'Itel S23 6.6" 128

In [ ]:
# shamlas

# def clean_data(data):
#     cleaned_data = []
    
#     for entry in data:
#         # Remove unwanted characters, keeping numbers and spaces
#         cleaned_entry = re.sub(r"[,'+\-]", " ", entry)  # Replace commas, single quotes, and plus signs with space
#         cleaned_entry = re.sub(r"\s+", " ", cleaned_entry)  # Replace multiple spaces with a single space
#         cleaned_entry = cleaned_entry.strip()  # Remove leading and trailing spaces
#         cleaned_data.append(cleaned_entry)
    
#     return cleaned_data

# # Clean the data
# cleaned_data = clean_data(data)

# cleaned_data

['XIAOMI Redmi A3 6.71" 3GB RAM 64GB (Dual SIM) 5000mAh Midnight Black (2 YRs WRTY)',
 'Tecno Spark 20 Android 13 6.6" 128GB 4GB RAM(4GB Extended) 50MP 5000mAh Gravity Black(1YR WRTY)',
 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Mystery White (1YR WRTY)',
 'Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual Sim) 50MP Camera 5000mAh Black',
 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Starry Black',
 'Samsung GALAXY A15 6.5 HD 4GB RAM 128GB ROM DUAL SIM 50MP 5000mAh Black',
 'Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual Sim) 50MP 5000mAh Black Smart Watch & Buds',
 'XIAOMI Redmi 13C 6.74" 8GB RAM 256GB ROM 50MP AI Triple Camera 5000mAh (Dual Sim) Midnight Black Free Gifts',
 'Tecno Spark 20 6.6 HD UP to 8GB RAM 128GB ROM (Dual Sim) 50MP 5000 mAh Cyber White',
 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Luxurious Gold',
 'Infinix Smart 8 6.6" HD 2GB RAM 64GB Android 13 (Dual sim) 5000mAh Shiny Gold',
 'Tecno Spark 20C 6.6 256GB HD

In [86]:
#separate sumsung entries from other entries in the name list by checking "name_duplicate"
samsung_phones=[]
other_phones=[]

for entry in name_list:
    if "Samsung" in entry['name_duplicate']:
        samsung_phones.append(entry)
    else:
        other_phones.append(entry)

samsung_phones, other_phones   

# # Get the length of the cleaned data
print(len(samsung_phones), len(other_phones))  # 3 and 3 respectivelysamsung_phones_data_length = len(samsung_phones)



2700 9300


In [62]:
# #shamlas
# # Separate Samsung entries from other brands
# samsung_phones = [spec for spec in data if "Samsung" in spec]
# other_phones = [spec for spec in data if "Samsung" not in spec]

# # Let us check the result
# samsung_phones, other_phones

In [ ]:

# Further cleaning to remove mathematical operators within the list
def clean_entries(entries):
    cleaned_entries = []
    for entry in entries:
        # Make a copy of the entry to avoid modifying the original
        cleaned_entry = entry.copy()
        
        # Remove mathematical operators from 'name_duplicate'
        cleaned_entry['name_duplicate'] = re.sub(r'[+\-]', '', entry["name_duplicate"])
        
        # Append the entire cleaned dictionary to the list
        cleaned_entries.append(cleaned_entry)
    
    return cleaned_entries

# Clean the samsung_phones list
samsung_phones = clean_entries(samsung_phones)

# Display the cleaned Samsung phone specifications
samsung_phones



[{'name': "Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM (Dual Sim) 50MP Camera, 5000mAh, Black",
  'name_duplicate': 'Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual Sim) 50MP Camera 5000mAh Black'},
 {'name': "Samsung GALAXY A15, 6.5'' HD+, 4GB RAM + 128GB ROM, DUAL SIM, 50MP,  5000mAh - Black",
  'name_duplicate': 'Samsung GALAXY A15 6.5 HD 4GB RAM 128GB ROM DUAL SIM 50MP 5000mAh Black'},
 {'name': "Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM (Dual Sim) 50MP, 5000mAh - Black + Smart Watch & Buds",
  'name_duplicate': 'Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual Sim) 50MP 5000mAh Black Smart Watch & Buds'},
 {'name': "Samsung Galaxy A15, 6.5'' Display, 4GB RAM + 128GB ROM (Dual Sim) 50MP,  5000mAh - Blue(2YRs WRTY)",
  'name_duplicate': 'Samsung Galaxy A15 6.5 Display 4GB RAM 128GB ROM (Dual Sim) 50MP 5000mAh Blue(2YRs WRTY)'},
 {'name': 'Samsung Galaxy A05s, 6.7", 64GB + 4GB (Dual SIM), 5000mAh, Black',
  'name_duplicate': 'Samsung Galaxy A05s 6.7" 64GB 4GB (Dual SIM) 5000mAh Bl

In [72]:
# #shamlas
# # Feature extraction from the samsung phones using regular expressions
# def extract_samsung_info(spec):
#     # Regular expressions for extracting relevant parts
#     name_pattern = r'^(Samsung\s+Galaxy\s+\w+\s*\w*)'  # Without the case-insensitivity flag
#     display_pattern = r'(\d+(\.\d+)?)\s*["\']'
#     ram_pattern = r'(\d+)GB RAM'
#     storage_pattern = r'(\d+GB)\s*ROM'
#     camera_pattern = r'(\d+MP)'
#     battery_pattern = r'(\d{4}mAh)'
#     sim_pattern = r'\(?(Dual\s+Sim|DUAL\s+SIM|Single\s+Sim|Single)\)?'
    
#     # Extract all the numbers that are followed by GB
#     gb_values = [int(m.group(1)) for m in re.finditer(r'(\d+)GB', spec)]

#     # Determine RAM and Storage based on size
#     ram = min(gb_values) if gb_values else None
#     storage = max(gb_values) if gb_values else None

#     # Updated color pattern to ensure it captures correctly
#     color_pattern = r'(?i)[-\s](Black|White|Blue|Red|Green|Yellow|Gold|Silver|Gray|Pink)(?=\s|\(|$)'
#     color_match = re.search(color_pattern, spec)
#     color = color_match.group(1).strip() if color_match else None

#     # Set a default display size if not found
#     display_size = re.search(display_pattern, spec).group(1) if re.search(display_pattern, spec) else "6.0"

#     # Use case-insensitive search for name
#     name_match = re.search(name_pattern, spec, re.IGNORECASE)
    
#     return {
#         "Name": name_match.group(0).strip() if name_match else None,
#         "Display Size": display_size,
#         "RAM": ram,
#         "Storage": storage,
#         "Camera": re.search(camera_pattern, spec).group(1) if re.search(camera_pattern, spec) else None,
#         "Battery": re.search(battery_pattern, spec).group(1) if re.search(battery_pattern, spec) else None,
#         "Color": color,
#         "Warranty": "Yes" if "WRTY" in spec else "No",
#         "Number of SIMs": 2 if re.search(sim_pattern, spec) and "Dual" in re.search(sim_pattern, spec).group(0) else 1
#     }

# # Extract information for Samsung phones

# samsung_phones_data = [extract_samsung_info(spec) for spec in samsung_phones,entry["name_duplicate"] ]

# # Create a DataFrame
# samsung_phones_df = pd.DataFrame(samsung_phones_data)

# # Display the DataFrame
# print("Samsung Phones:")
# samsung_phones_df.head(10)

In [102]:

# Feature extraction from the samsung phones using regular expressions
def extract_samsung_info(spec):
    # Regular expressions for extracting relevant parts
    name_pattern = r'^(Samsung\s+Galaxy\s+\w+\s*\w*)'
    display_pattern = r'(\d+(\.\d+)?)\s*["\']'
    ram_pattern = r'(\d+)GB RAM'
    storage_pattern = r'(\d+GB)\s*ROM'
    camera_pattern = r'(\d+MP)'
    battery_pattern = r'(\d{4}mAh)'
    sim_pattern = r'\(?(Dual\s+Sim|DUAL\s+SIM|Single\s+Sim|Single)\)?'
    
    # Extract all the numbers that are followed by GB
    gb_values = [int(m.group(1)) for m in re.finditer(r'(\d+)GB', spec)]

    # Determine RAM and Storage based on size
    ram = min(gb_values) if gb_values else None
    storage = max(gb_values) if gb_values else None

    # Updated color pattern to ensure it captures correctly
    color_pattern = r'(?i)[-\s](Black|White|Blue|Red|Green|Yellow|Gold|Silver|Gray|Pink)(?=\s|\(|$)'
    color_match = re.search(color_pattern, spec)
    color = color_match.group(1).strip() if color_match else None

    # Set a default display size if not found
    display_size = re.search(display_pattern, spec).group(1) if re.search(display_pattern, spec) else "6.0"

    # Use case-insensitive search for name
    name_match = re.search(name_pattern, spec, re.IGNORECASE)
    
    return {
        "Phone_Name": name_match.group(0).strip() if name_match else None,
        "Display Size": display_size,
        "RAM": ram,
        "Storage": storage,
        "Camera": re.search(camera_pattern, spec).group(1) if re.search(camera_pattern, spec) else None,
        "Battery": re.search(battery_pattern, spec).group(1) if re.search(battery_pattern, spec) else None,
        "Color": color,
        "Warranty": "Yes" if "WRTY" in spec else "No",
        "Number of SIMs": 2 if re.search(sim_pattern, spec) and "Dual" in re.search(sim_pattern, spec).group(0) else 1
    }

# Sample data for samsung_phones
# samsung_phones = [
#     {'name': 'Samsung Galaxy A14, 6.6", 4GB RAM + 64GB (Dual SIM), 5000mAh, Black (1 YR WRTY)',
#      'name_duplicate': 'Samsung Galaxy A14, 6.6", 4GB RAM + 64GB (Dual SIM), 5000mAh, Black (1 YR WRTY)'},
#     {'name': 'Samsung Galaxy S22, 6.1", 8GB RAM - 128GB (Single SIM), 3700mAh, Phantom Black',
#      'name_duplicate': 'Samsung Galaxy S22, 6.1", 8GB RAM - 128GB (Single SIM), 3700mAh, Phantom Black'}
# ]

# Extract information for Samsung phones and expand the dictionaries
for entry in samsung_phones:
    # Extract info using name_duplicate
    extracted_info = extract_samsung_info(entry["name_duplicate"])
    # Update the existing entry with the extracted information
    entry.update(extracted_info)

# Create a DataFrame
samsung_phones_df = pd.DataFrame(samsung_phones)

# Display the DataFrame
print("Samsung Phones:")
print(samsung_phones_df.head())
print(samsung_phones_df.shape)
print(type(samsung_phones_df))

Samsung Phones:
                                                name  \
0  Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM ...   
1  Samsung GALAXY A15, 6.5'' HD+, 4GB RAM + 128GB...   
2  Samsung Galaxy A05, 6.7'' 4GB RAM + 128GB ROM ...   
3  Samsung Galaxy A15, 6.5'' Display, 4GB RAM + 1...   
4  Samsung Galaxy A05s, 6.7", 64GB + 4GB (Dual SI...   

                                      name_duplicate             Phone_Name  \
0  Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual...   Samsung Galaxy A05 6   
1  Samsung GALAXY A15 6.5 HD 4GB RAM 128GB ROM DU...   Samsung GALAXY A15 6   
2  Samsung Galaxy A05 6.7 4GB RAM 128GB ROM (Dual...   Samsung Galaxy A05 6   
3  Samsung Galaxy A15 6.5 Display 4GB RAM 128GB R...   Samsung Galaxy A15 6   
4  Samsung Galaxy A05s 6.7" 64GB 4GB (Dual SIM) 5...  Samsung Galaxy A05s 6   

  Display Size  RAM  Storage Camera  Battery  Color Warranty  Number of SIMs  
0          6.0    4      128   50MP  5000mAh  Black       No               2  
1          6.0

In [71]:
# #shamlas
# # Cleaning up other phones
# def clean_entries(entries):
#     cleaned_entries = []
#     for entry in entries:
#         # Remove mathematical operators
#         cleaned_entry = re.sub(r'[+\-]', '', entry)
#         cleaned_entries.append(cleaned_entry)
#     return cleaned_entries

# # Clean the list
# other_phones = clean_entries(other_phones)

# other_phones

In [ ]:
import re

# Further cleaning to remove mathematical operators within the other phones list
def clean_entries(entries):
    cleaned_entries = []
    for entry in entries:
        # Make a copy of the entry to avoid modifying the original
        cleaned_entry = entry.copy()
        
        # Remove mathematical operators from 'name_duplicate'
        cleaned_entry['name_duplicate'] = re.sub(r'[+\-]', '', entry["name_duplicate"])
        
        # Append the entire cleaned dictionary to the list
        cleaned_entries.append(cleaned_entry)
    
    return cleaned_entries

# Clean the samsung_phones list
other_phones = clean_entries(other_phones)

# Display the cleaned Samsung phone specifications
other_phones


[{'name': 'XIAOMI Redmi A3, 6.71", 3GB RAM + 64GB (Dual SIM), 5000mAh, Midnight Black  (2 YRs WRTY)',
  'name_duplicate': 'XIAOMI Redmi A3 6.71" 3GB RAM 64GB (Dual SIM) 5000mAh Midnight Black (2 YRs WRTY)'},
 {'name': 'Tecno Spark 20, Android 13, 6.6", 128GB + 4GB RAM(4GB Extended), 50MP, 5000mAh, Gravity Black(1YR WRTY)',
  'name_duplicate': 'Tecno Spark 20 Android 13 6.6" 128GB 4GB RAM(4GB Extended) 50MP 5000mAh Gravity Black(1YR WRTY)'},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Mystery White (1YR WRTY)',
  'name_duplicate': 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Mystery White (1YR WRTY)'},
 {'name': 'Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (Dual SIM), 4G, 5000mAh - Starry Black',
  'name_duplicate': 'Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual SIM) 4G 5000mAh Starry Black'},
 {'name': 'XIAOMI Redmi 13C, 6.74"  8GB RAM + 256GB ROM, 50MP AI Triple Camera, 5000mAh (Dual Sim) - Midnight Black + Free Gifts',
  'name_

In [73]:
# #shamlas
# # Feature extraction from the list of other phones
# def extract_other_phones_info(spec):
#     # Regular expressions for extracting relevant parts
#     name_pattern = r'^(XIAOMI|Tecno|Infinix|Itel|Oale)\s+(\w+\s*\w*)'
#     display_pattern = r'(\d+(\.\d+)?)\s*["\']'
#     ram_pattern = r'(\d+)GB\s*RAM'
#     storage_pattern = r'(\d+GB|(\d+MB))\s*(ROM|Storage)'
#     camera_pattern = r'(\d+MP)'
#     battery_pattern = r'(\d{4}mAh|\d{4}MAh)'
#     sim_pattern = r'\(?(Dual\s+SIM|Single\s+SIM|Single)\)?'
    
#     # Extract RAM and Storage
#     gb_values = [int(m.group(1)) for m in re.finditer(r'(\d+)GB', spec)]
    
#     ram = min(gb_values) if gb_values else None
#     storage = max(gb_values) if gb_values else None

#     # Extract color with a broader pattern
#     color_pattern = r'(?i)[-\s](Black|White|Blue|Red|Green|Yellow|Gold|Silver|Gray|Pink)(?=\s|\(|$)'
#     color_match = re.search(color_pattern, spec)
#     color = color_match.group(1).strip() if color_match else None

#     # Set a default display size if not found
#     display_size = re.search(display_pattern, spec).group(1) if re.search(display_pattern, spec) else "6.0"

#     # Extract name, ensuring it doesn't include the display size
#     name_match = re.search(name_pattern, spec, re.IGNORECASE)
#     if name_match:
#         name = name_match.group(0).strip()
#         # Remove any trailing numbers or characters (like display sizes)
#         name = re.sub(r'\s*\d*$', '', name).strip()
#     else:
#         name = None

#     return {
#         "Name": name,
#         "Display Size": display_size,
#         "RAM": ram,
#         "Storage": storage,
#         "Camera": re.search(camera_pattern, spec).group(1) if re.search(camera_pattern, spec) else None,
#         "Battery": re.search(battery_pattern, spec).group(1) if re.search(battery_pattern, spec) else None,
#         "Color": color,
#         "Warranty": "Yes" if "WRTY" in spec else "No",
#         "Number of SIMs": 2 if re.search(sim_pattern, spec) and "Dual" in re.search(sim_pattern, spec).group(0) else 1
#     }

# # Extract information for other phones
# other_phones_data = [extract_other_phones_info(spec) for spec in other_phones]

# # Create a DataFrame
# other_phones_df = pd.DataFrame(other_phones_data)

# # Display the DataFrame
# print("Other Phones:")
# other_phones_df.head(20)

In [101]:
# Feature extraction from the other_ phones using regular expressions
def extract_other_phones_info(spec):
    # Regular expressions for extracting relevant parts
    name_pattern = r'^(Samsung\s+Galaxy\s+\w+\s*\w*)'
    display_pattern = r'(\d+(\.\d+)?)\s*["\']'
    ram_pattern = r'(\d+)GB RAM'
    storage_pattern = r'(\d+GB)\s*ROM'
    camera_pattern = r'(\d+MP)'
    battery_pattern = r'(\d{4}mAh)'
    sim_pattern = r'\(?(Dual\s+Sim|DUAL\s+SIM|Single\s+Sim|Single)\)?'
    
    # Extract all the numbers that are followed by GB
    gb_values = [int(m.group(1)) for m in re.finditer(r'(\d+)GB', spec)]

    # Determine RAM and Storage based on size
    ram = min(gb_values) if gb_values else None
    storage = max(gb_values) if gb_values else None

    # Updated color pattern to ensure it captures correctly
    color_pattern = r'(?i)[-\s](Black|White|Blue|Red|Green|Yellow|Gold|Silver|Gray|Pink)(?=\s|\(|$)'
    color_match = re.search(color_pattern, spec)
    color = color_match.group(1).strip() if color_match else None

    # Set a default display size if not found
    display_size = re.search(display_pattern, spec).group(1) if re.search(display_pattern, spec) else "6.0"

    # Use case-insensitive search for name
    name_match = re.search(name_pattern, spec, re.IGNORECASE)
    
    return {
        "Phone_Name": name_match.group(0).strip() if name_match else None,
        "Display Size": display_size,
        "RAM": ram,
        "Storage": storage,
        "Camera": re.search(camera_pattern, spec).group(1) if re.search(camera_pattern, spec) else None,
        "Battery": re.search(battery_pattern, spec).group(1) if re.search(battery_pattern, spec) else None,
        "Color": color,
        "Warranty": "Yes" if "WRTY" in spec else "No",
        "Number of SIMs": 2 if re.search(sim_pattern, spec) and "Dual" in re.search(sim_pattern, spec).group(0) else 1
    }


# Extract information for Samsung phones and expand the dictionaries
for entry in other_phones:
    # Extract info using name_duplicate
    extracted_info = extract_other_phones_info(entry["name_duplicate"])
    # Update the existing entry with the extracted information
    entry.update(extracted_info)

# Create a DataFrame
other_phones_df = pd.DataFrame(other_phones)

# Display the DataFrame
print("Other_Phones:")
print(other_phones_df.head())
print(other_phones_df.shape)
print(type(other_phones_df)) # Check the data type of other_phones_df.columns)

Other_Phones:
                                                name  \
0  XIAOMI Redmi A3, 6.71", 3GB RAM + 64GB (Dual S...   
1  Tecno Spark 20, Android 13, 6.6", 128GB + 4GB ...   
2  Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (...   
3  Itel S23 6.6", 128GB + 4GB RAM, 50MP Camera, (...   
4  XIAOMI Redmi 13C, 6.74"  8GB RAM + 256GB ROM, ...   

                                      name_duplicate Phone_Name Display Size  \
0  XIAOMI Redmi A3 6.71" 3GB RAM 64GB (Dual SIM) ...       None         6.71   
1  Tecno Spark 20 Android 13 6.6" 128GB 4GB RAM(4...       None          6.6   
2  Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual ...       None          6.6   
3  Itel S23 6.6" 128GB 4GB RAM 50MP Camera (Dual ...       None          6.6   
4  XIAOMI Redmi 13C 6.74" 8GB RAM 256GB ROM 50MP ...       None         6.74   

   RAM  Storage Camera  Battery  Color Warranty  Number of SIMs  
0    3       64   None  5000mAh  Black      Yes               1  
1    4      128   50MP  5000mAh  Bla

In [173]:
#check for duplicates in samsumg_phones_df and other_phones_df based on phone name
print(samsung_phones_df.duplicated(subset="name").sum())
print(other_phones_df.duplicated(subset="name").sum())

2691
9269


In [ ]:
print(samsung_phones_df.duplicated().sum())
print(other_phones_df.duplicated().sum())

In [106]:
print(samsung_phones_df.columns)
print(other_phones_df.columns)


Index(['name', 'name_duplicate', 'Phone_Name', 'Display Size', 'RAM',
       'Storage', 'Camera', 'Battery', 'Color', 'Warranty', 'Number of SIMs'],
      dtype='object')
Index(['name', 'name_duplicate', 'Phone_Name', 'Display Size', 'RAM',
       'Storage', 'Camera', 'Battery', 'Color', 'Warranty', 'Number of SIMs'],
      dtype='object')


In [168]:
#merge the samsung df and other_phones_df
smartphones_df = pd.concat([samsung_phones_df, other_phones_df]).sort_values("name")

# Display the merged DataFrame
smartphones_df.head()
print(smartphones_df.shape)
print(smartphones_df.columns)

(12000, 11)
Index(['name', 'name_duplicate', 'Phone_Name', 'Display Size', 'RAM',
       'Storage', 'Camera', 'Battery', 'Color', 'Warranty', 'Number of SIMs'],
      dtype='object')


In [169]:
#remane the name column in the smarphones_df
smartphones_df.rename(columns={"name":"Name"}, inplace=True)
print(smartphones_df.columns)
print(df.columns)

Index(['Name', 'name_duplicate', 'Phone_Name', 'Display Size', 'RAM',
       'Storage', 'Camera', 'Battery', 'Color', 'Warranty', 'Number of SIMs'],
      dtype='object')
Index(['Name', 'Price', 'Old Price', 'Discount', 'Rating', 'Number of Reviews',
       'Search Ranking', 'Name_duplicate'],
      dtype='object')


In [171]:
print("Duplicates in smartphones_df:")
print(smartphones_df.duplicated().sum())

print("Duplicates in df:")
print(df.duplicated().sum())


Duplicates in smartphones_df:
11960
Duplicates in df:
0


In [170]:
#merge df and smartphones_df by comparing the  names column
complete_df = pd.merge(df, smartphones_df, on='Name', how='left')
complete_df.head()
print(complete_df.shape)

(3600000, 18)


In [158]:
#drop name_duplicate and Name_duplicate columns
complete_df.drop(['name_duplicate', 'Name_duplicate'], axis=1, inplace=True)
complete_df.head()

,Name,Price,Old Price,Discount,Rating,Number of Reviews,Search Ranking,Phone_Name,Display Size,RAM,Storage,Camera,Battery,Color,Warranty,Number of SIMs
0,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,64,None,5000mAh,Black,Yes,1
1,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,64,None,5000mAh,Black,Yes,1
2,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,64,None,5000mAh,Black,Yes,1
3,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,64,None,5000mAh,Black,Yes,1
4,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,64,None,5000mAh,Black,Yes,1


#### Further feature exttaction in the remaining columns i.e Rank and Reviews

In [162]:
#extract Number of reviews from the number of reviews column
complete_df["Total_Reviews"] = complete_df["Number of Reviews"].str.extract(r'\((.*?)\)', expand=False)
#extract  rank from the searching rank column
complete_df["Rank"] = complete_df["Search Ranking"].str.extract(r"Rank (\d+)", expand=False)
#extract the page number from the searching rank column
complete_df["Page_Number"] = complete_df["Search Ranking"].str.extract(r"Page (\d+)", expand=False).str.get(0)
# Extracting the rating numerator
complete_df["Rating_Numerator"] = complete_df["Rating"].str.extract(r'(\d+(\.\d+)?)\s*out\s+of', expand=False)[0]
# Extracting the rating denominator (the number after 'out of')
complete_df["Rating_Denominator"] = complete_df["Rating"].str.extract(r'out\s+of\s+(\d+)', expand=False)

complete_df.head()

,Name,Price,Old Price,Discount,Rating,Number of Reviews,Search Ranking,Phone_Name,Display Size,RAM,...,Camera,Battery,Color,Warranty,Number of SIMs,Total_Reviews,Rank,Page_Number,Rating_Numerator,Rating_Denominator
0,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,...,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
1,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,...,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
2,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,...,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
3,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,...,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
4,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,4.1 out of 5,4.1 out of 5(220),"Page 1, Rank 1",None,6.71,3,...,None,5000mAh,Black,Yes,1,220,1,1,4.1,5


#### Lets drop the redudant columns i.e the number of reviews column and the search ranking

In [163]:
#drop the redudant columns
complete_df.drop(['Number of Reviews', 'Search Ranking', 'Rating'], axis=1, inplace=True)
complete_df.head()

,Name,Price,Old Price,Discount,Phone_Name,Display Size,RAM,Storage,Camera,Battery,Color,Warranty,Number of SIMs,Total_Reviews,Rank,Page_Number,Rating_Numerator,Rating_Denominator
0,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,None,6.71,3,64,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
1,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,None,6.71,3,64,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
2,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,None,6.71,3,64,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
3,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,None,6.71,3,64,None,5000mAh,Black,Yes,1,220,1,1,4.1,5
4,"XIAOMI Redmi A3, 6.71"", 3GB RAM + 64GB (Dual S...","KSh 11,000",NaN,NaN,None,6.71,3,64,None,5000mAh,Black,Yes,1,220,1,1,4.1,5


## Data cleaning

In [166]:
#shape of complete_df
complete_df.shape

(3600000, 18)

In [165]:
#check the missing values in the complete_df
complete_df.isnull().sum()

Name                        0
Price                       0
Old Price             1169700
Discount              1169700
Phone_Name            2790000
Display Size                0
RAM                         0
Storage                     0
Camera                 990000
Battery                810000
Color                  720000
Warranty                    0
Number of SIMs              0
Total_Reviews           90000
Rank                        0
Page_Number                 0
Rating_Numerator        90000
Rating_Denominator      90000
dtype: int64

In [176]:
#drop missing values
complete_df.dropna(inplace=True)
complete_df.shape

(630000, 18)

In [177]:
complete_df.isnull().sum()

Name                 0
Price                0
Old Price            0
Discount             0
Rating               0
Number of Reviews    0
Search Ranking       0
Name_duplicate       0
name_duplicate       0
Phone_Name           0
Display Size         0
RAM                  0
Storage              0
Camera               0
Battery              0
Color                0
Warranty             0
Number of SIMs       0
dtype: int64

In [178]:
#dropm duplicates
complete_df.drop_duplicates(inplace=True)
complete_df.shape

(2100, 18)